In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Load the data
atomicdescriports = pd.read_csv('./data/descriptors_template.csv', index_col=0)

fp = open("./data/list_structural_descriptors.txt", "r")
descdetails = {}
for l in fp:
    k = l.split(":")[0].replace("\"", "")
    v = l.split(":")[1].replace("\n", "").replace("#", "")
    descdetails[k] = v
fp.close()

print("Number of descriptors: ", len(atomicdescriports.columns))

In [ ]:
nonvalues = ['label_structure', 'composition_pretty', 'A-site', 'B-site']
for c in nonvalues:
    print(atomicdescriports[c].value_counts())
    print(atomicdescriports[c].unique())

In [ ]:
#remove constant columns
for c in atomicdescriports.columns:    
    if len(atomicdescriports[c].unique()) == 1:
        print("Removing constant column: ", c)
        atomicdescriports.drop(c, axis=1, inplace=True)

In [ ]:
corrcut = 0.90
tocrrolate = {}
for c in atomicdescriports.columns:
    if c not in nonvalues:
        tocrrolate[c] = atomicdescriports[c].astype(np.float64).values

# Correlation matrix
toremove = set()
basicdescr = set()
tocrrolate = pd.DataFrame(tocrrolate)
corr = tocrrolate.corr().abs()
for c in corr.columns:
    print(c)
    if c not in toremove:
        basicdescr.add(c)
        for cc in corr.columns:
            if corr[c][cc] > corrcut and c != cc and  \
                cc not in basicdescr:
                print("\t %20s %7.3f"%(cc, corr[c][cc]))
                toremove.add(cc)

In [ ]:
print("Removing columns: ")
for c in toremove:
    print("  ", c)
    tocrrolate.drop(c, axis=1, inplace=True)

# Plot the correlation matrix
corr = tocrrolate.corr()
plt.figure(figsize=(20, 20))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f")

In [ ]:
atomic_descriptors = {}
print("Atomic features to keep: ")
for c in tocrrolate.columns:
    print("  ", c)
    atomic_descriptors[c] = atomicdescriports[c].values
    

In [ ]:
materialdata = pd.read_excel('./data/materialdata.xlsx')

# Remove rows with NaN in d33
materialdata = materialdata[~materialdata["d33"].isna()]
# Remove rows with d33 values between -5 and 100
materialdata = materialdata[(materialdata["d33"] > -5) & (materialdata["d33"] < 100)]

print(materialdata.shape)

In [ ]:
Y = materialdata["d33"].values
featurestouse = ["formal_charge_A-site", \
    "formal_charge_B-site", "octahedra_volume_min", \
    "octahedra_volume_max", "octahedra_volume_avg", \
    "octahedra_meanangle_axis_min" , \
    "octahedra_meanangle_axis_max" , \
    "octahedra_meanangle_axis_avg" , \
    "tilt_BOB_ip_min" ,\
    "tilt_BOB_ip_max" , \
    "tilt_BOB_ip_avg", \
    "tilt_BOB_oop_min", \
    "tilt_BOB_oop_max", \
    "tilt_BOB_oop_avg", \
    "spageGroup_no" ,\
    "lattice_a", \
    "lattice_b", \
    "lattice_c", \
    "lattice_alfa", \
    "lattice_beta" ,\
    "lattice_gamma" ,\
    "volume_uc" ,\
    "volume_uc_per_atom" ,\
    "volume_ratio_ucVSoctahedra" ,\
    "tolerance_factor" ,\
    "ratio_outVSinplaneAVG" ,\
    "ratio_outVSinplanemin" ,\
    "ratio_outVSinplanemax" ,\
    "bond_lengthAA_min" ,\
    "bond_lengthAA_max" ,\
    "bond_lengthAA_avg" ,\
    "bond_lengthAB_min" ,\
    "bond_lengthAB_max" ,\
    "bond_lengthAB_avg" ,\
    "bond_lengthAO_min" ,\
    "bond_lengthAO_max" ,\
    "bond_lengthAO_avg" ,\
    "bond_lengthBO_min" ,\
    "bond_lengthBO_max" ,\
    "bond_lengthBO_avg" ,\
    "bond_lengthBB_min" ,\
    "bond_lengthBB_max" ,\
    "bond_lengthBB_avg"]
X = materialdata[featurestouse]

#remove high correlated values from X
toremove = set()
basicdescr = set()
tocrrolate = pd.DataFrame(X)
corr = tocrrolate.corr().abs()
for c in corr.columns:
    print(c)
    if c not in toremove:
        basicdescr.add(c)
        for cc in corr.columns:
            if corr[c][cc] > corrcut and c != cc and  \
                cc not in basicdescr:
                print("\t %20s %7.3f"%(cc, corr[c][cc]))
                toremove.add(cc)

feattouse = []
print("Featurs to use: ")
for c in basicdescr:
    print("  ", c)
    feattouse.append(c)

In [ ]:
Y = materialdata["d33"].values
X = materialdata[feattouse]
print(X.shape)
print(Y.shape)

In [ ]:
#build a PLS model incresing the number of components   
import matplotlib.pyplot as plt
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import mean_squared_error

rmses = []
looormses = []
r2s = []
ncomps = []
for n in range(1, X.shape[1]):
    ncomps.append(n)
    pls = PLSRegression(n_components=n)
    pls.fit(X, Y)
    Y_pred = pls.predict(X)
    r2s.append(pls.score(X, Y))
    rmse = mean_squared_error(Y, Y_pred, squared=False)
    rmses.append(rmse)

    # use and compute Leave one out cross validation

    loo = LeaveOneOut()
    Y_pred = cross_val_predict(pls, X, Y, cv=loo)
    rmse = mean_squared_error(Y, Y_pred, squared=False) 
    looormses.append(rmse)


plt.figure(figsize=(10, 5))
plt.plot(ncomps, r2s, label="R2")
plt.xlabel("Number of components")
plt.ylabel("RMSE")
plt.legend()
plt.xticks(ncomps)
plt.show()

plt.clf()
plt.figure(figsize=(10, 5))
plt.plot(ncomps, rmses, label="RMSE")
plt.plot(ncomps, looormses, label="LOO RMSE")
plt.xlabel("Number of components")
plt.ylabel("RMSE")
plt.legend()
plt.xticks(ncomps)
plt.show()

In [ ]:
ncomptouse = 11
pls = PLSRegression(n_components=ncomptouse)
pls.fit(X, Y)
Y_pred = pls.predict(X)
rmse = mean_squared_error(Y, Y_pred, squared=False)
print("RMSE: ", rmse)
print("R2: ", pls.score(X, Y))
# plot the scatterplot
plt.clf()
plt.figure(figsize=(10, 5))
plt.scatter(Y, Y_pred)
plt.xlabel("Experimental d33")
plt.ylabel("Predicted d33")
plt.show()

In [ ]:
# build a cluster model
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

sil = []
kmax = 10
for k in range(2, kmax):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(X)
    labels = kmeans.labels_
    sil.append(silhouette_score(X, labels, metric = 'euclidean'))

plt.clf()
plt.figure(figsize=(10, 5))
plt.plot(range(2, kmax), sil)
plt.xlabel("Number of clusters")
plt.ylabel("Silhouette score")
plt.show()


In [ ]:
# BUILD A CLUSTER MODEL
k = 3
kmeans = KMeans(n_clusters=k)
kmeans.fit(X)
labels = kmeans.labels_
print(labels)
materialdata["cluster"] = labels

# plot THE CLUSTER MODEL
plt.clf()
plt.figure(figsize=(10, 5))
plt.scatter(materialdata["d33"], materialdata["cluster"])
plt.xlabel("d33")
plt.ylabel("Cluster")
plt.show()
